In [1]:
import streamlit as st
import pandas as pd
import pydeck as pdk
import hashlib
import glob
import streamlit.components.v1 as components
import plotly.graph_objects as go
import matplotlib.colors as mcolors

In [2]:
# 📂 Ruta donde están los archivos
ruta_archivos = "reportes_clientes/*.xlsx"

# 🧾 Lista de todos los archivos que coincidan
archivos = glob.glob(ruta_archivos)

# 📊 Leer y concatenar todos los archivos (solo hoja 'Resumen ruta')
dfs = []

for archivo in archivos:
    try:
        df_temp = pd.read_excel(archivo, sheet_name="Reporte diario cliente")
        df_temp["Archivo"] = archivo  # opcional: para saber de qué archivo viene
        dfs.append(df_temp)
    except ValueError:
        print(f"⚠️ La hoja 'Resumen ruta' no se encontró en {archivo}")
    except Exception as e:
        print(f"❌ Error al leer {archivo}: {e}")

# Concatenar si hay data
if dfs:
    df_mrk = pd.concat(dfs, ignore_index=True)
    print("✅ Archivos combinados correctamente:", len(dfs))
else:
    print("⚠️ No se pudo leer ningún archivo válido.")

✅ Archivos combinados correctamente: 40


In [3]:
df_mrk

,Zona,Región,Gerencia,Jefatura,Ruta,Id cliente,Cliente,Canal,GEC,Fecha,...,Latitud del maestro,Longitud del maestro,Tiempo de atencion,Geoeficiencia fuera de frecuencia,Geoefectividad fuera de frecuencia,Fuera de frecuencia,Pedido Digital,Pedido Omnicanal,Planeado Digital,Archivo
0,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901029,ABTS HERRERA,ABARROTES Y MISCELANEAS,PLATA,2025-09-09,...,17.834357,-97.788940,00:17:49,1,1,0,0,1,1,reportes_clientes/IndicadoresReport_2025-09-09...
1,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901031,MISC ERIKA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-11,...,18.018685,-97.896553,00:02:11,1,1,0,0,1,1,reportes_clientes/IndicadoresReport_2025-09-09...
2,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901033,AGUSTIN SOSA LEZAMA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-11,...,18.019240,-97.898173,00:00:41,1,0,1,0,0,0,reportes_clientes/IndicadoresReport_2025-09-09...
3,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901034,SOSITA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-09,...,18.021135,-97.899470,00:12:45,1,1,0,1,1,1,reportes_clientes/IndicadoresReport_2025-09-09...
4,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901041,MISC TONITO,ABARROTES Y MISCELANEAS,BRONCE,2025-09-10,...,18.082260,-97.918300,00:01:01,1,1,0,0,1,1,reportes_clientes/IndicadoresReport_2025-09-09...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11015236,CENTRO-PACIFICO,ACAPULCO,HR01,HR1,HR0118,1950086095,MISCELANEA LOS ANGELES,ABARROTES Y MISCELANEAS,BRONCE,2025-09-24,...,16.911502,-99.802992,00:00:04,1,1,0,0,0,1,reportes_clientes/IndicadoresReport_2025-09-17...
11015237,CENTRO-PACIFICO,ACAPULCO,HR01,HR1,HR0093,1950086096,MISC NICO,ABARROTES Y MISCELANEAS,BRONCE,2025-09-24,...,16.929762,-99.833150,00:05:55,1,1,0,0,1,1,reportes_clientes/IndicadoresReport_2025-09-17...
11015238,CENTRO-PACIFICO,COSTAS,HN01,HN2,HN0072,1950086106,ABARROTES EL LLANITO,HOGAR CON VENTA,BRONCE,2025-09-22,...,17.162812,-100.388523,00:06:24,1,1,0,0,1,1,reportes_clientes/IndicadoresReport_2025-09-17...
11015239,CENTRO-PACIFICO,CHILAPA-CHILPANCINGO,HV01,HV2,HV0139,1950086116,DULCERIA SOL,ABARROTES Y MISCELANEAS,BRONCE,2025-09-24,...,17.563190,-99.505512,00:00:13,1,1,0,0,0,1,reportes_clientes/IndicadoresReport_2025-09-17...


In [4]:
from datetime import time

# 🕒 1. Detectar columnas con "(hr)"
cols_hr = [col for col in df_mrk.columns if '(hr)' in col]
print("Columnas a convertir:", cols_hr)

# 🧭 2. Normalizar y convertir
for col in cols_hr:
    # Si algún valor es datetime.time, conviértelo a string "HH:MM:SS"
    df_mrk[col] = df_mrk[col].apply(
        lambda x: x.strftime('%H:%M:%S') if isinstance(x, time) else x
    )
    # Luego convertir a timedelta
    df_mrk[col] = pd.to_timedelta(df_mrk[col], errors='coerce')

# 📊 Verificar
df_mrk.info()

Columnas a convertir: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11015241 entries, 0 to 11015240
Data columns (total 31 columns):
 #   Column                                 Dtype         
---  ------                                 -----         
 0   Zona                                   object        
 1   Región                                 object        
 2   Gerencia                               object        
 3   Jefatura                               object        
 4   Ruta                                   object        
 5   Id cliente                             int64         
 6   Cliente                                object        
 7   Canal                                  object        
 8   GEC                                    object        
 9   Fecha                                  datetime64[ns]
 10  Pedido                                 int64         
 11  Distancia de llegada al cliente (mts)  float64       
 12  Distancia de salida al client

In [5]:
df_mrk.head()

,Zona,Región,Gerencia,Jefatura,Ruta,Id cliente,Cliente,Canal,GEC,Fecha,...,Latitud del maestro,Longitud del maestro,Tiempo de atencion,Geoeficiencia fuera de frecuencia,Geoefectividad fuera de frecuencia,Fuera de frecuencia,Pedido Digital,Pedido Omnicanal,Planeado Digital,Archivo
0,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901029,ABTS HERRERA,ABARROTES Y MISCELANEAS,PLATA,2025-09-09,...,17.834357,-97.788940,00:17:49,1,1,0,0,1,1,reportes_clientes/IndicadoresReport_2025-09-09...
1,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901031,MISC ERIKA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-11,...,18.018685,-97.896553,00:02:11,1,1,0,0,1,1,reportes_clientes/IndicadoresReport_2025-09-09...
2,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901033,AGUSTIN SOSA LEZAMA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-11,...,18.019240,-97.898173,00:00:41,1,0,1,0,0,0,reportes_clientes/IndicadoresReport_2025-09-09...
3,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901034,SOSITA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-09,...,18.021135,-97.899470,00:12:45,1,1,0,1,1,1,reportes_clientes/IndicadoresReport_2025-09-09...
4,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901041,MISC TONITO,ABARROTES Y MISCELANEAS,BRONCE,2025-09-10,...,18.082260,-97.918300,00:01:01,1,1,0,0,1,1,reportes_clientes/IndicadoresReport_2025-09-09...


In [6]:
df_catalogo = pd.read_csv('catalogo.csv', encoding='utf-8')
df_catalogo = df_catalogo[['Ruta','Jefatura','Tipo','Descripción Tipo','TPV','Región Comercial_Act 2026','Nombre UO']]

df_mrk_full = df_mrk.merge(how='left', right=df_catalogo, left_on='Ruta', right_on='Ruta')
df_mrk_full.head()

,Zona,Región,Gerencia,Jefatura_x,Ruta,Id cliente,Cliente,Canal,GEC,Fecha,...,Pedido Digital,Pedido Omnicanal,Planeado Digital,Archivo,Jefatura_y,Tipo,Descripción Tipo,TPV,Región Comercial_Act 2026,Nombre UO
0,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901029,ABTS HERRERA,ABARROTES Y MISCELANEAS,PLATA,2025-09-09,...,0,1,1,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan
1,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901031,MISC ERIKA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-11,...,0,1,1,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan
2,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901033,AGUSTIN SOSA LEZAMA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-11,...,0,0,0,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan
3,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901034,SOSITA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-09,...,1,1,1,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan
4,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901041,MISC TONITO,ABARROTES Y MISCELANEAS,BRONCE,2025-09-10,...,0,1,1,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan


In [7]:
df_mrk_cp = df_mrk_full[df_mrk_full['Zona'] == 'CENTRO-PACIFICO']
df_mrk_cp = df_mrk_cp[df_mrk_cp['Tipo'].isin(['PR', 'PCOP', 'FC', 'MA', 'PRTDC'])]
df_mrk_cp = df_mrk_cp.rename(columns={'Fecha': 'Fecha inicio'})

df_mrk_cp['Región'] = df_mrk_cp['Región'].where(
~df_mrk_cp['Tipo'].isin(['MA', 'PRTDC'])
).fillna(df_mrk_cp['Tipo'].replace({'MA': 'MAYORISTAS', 'PRTDC': 'MODERNO'}))
df_mrk_cp['Semana'] = df_mrk_cp['Fecha inicio'].dt.isocalendar().week

df_mrk_cp

,Zona,Región,Gerencia,Jefatura_x,Ruta,Id cliente,Cliente,Canal,GEC,Fecha inicio,...,Pedido Omnicanal,Planeado Digital,Archivo,Jefatura_y,Tipo,Descripción Tipo,TPV,Región Comercial_Act 2026,Nombre UO,Semana
0,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901029,ABTS HERRERA,ABARROTES Y MISCELANEAS,PLATA,2025-09-09,...,1,1,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan,37
1,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901031,MISC ERIKA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-11,...,1,1,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan,37
2,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901033,AGUSTIN SOSA LEZAMA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-11,...,0,0,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan,37
3,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901034,SOSITA,ABARROTES Y MISCELANEAS,BRONCE,2025-09-09,...,1,1,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan,37
4,CENTRO-PACIFICO,HUAJUAPAN,HJ01,HJ1,HJ0008,200901041,MISC TONITO,ABARROTES Y MISCELANEAS,BRONCE,2025-09-10,...,1,1,reportes_clientes/IndicadoresReport_2025-09-09...,HJ1,PR,Preventa Comercial,ZPV,Huajuapan,Huajuapan,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11015236,CENTRO-PACIFICO,ACAPULCO,HR01,HR1,HR0118,1950086095,MISCELANEA LOS ANGELES,ABARROTES Y MISCELANEAS,BRONCE,2025-09-24,...,0,1,reportes_clientes/IndicadoresReport_2025-09-17...,HR1,PR,Preventa Comercial,ZPV,Acapulco,Renacimiento,39
11015237,CENTRO-PACIFICO,ACAPULCO,HR01,HR1,HR0093,1950086096,MISC NICO,ABARROTES Y MISCELANEAS,BRONCE,2025-09-24,...,1,1,reportes_clientes/IndicadoresReport_2025-09-17...,HR1,PR,Preventa Comercial,ZPV,Acapulco,Renacimiento,39
11015238,CENTRO-PACIFICO,COSTAS,HN01,HN2,HN0072,1950086106,ABARROTES EL LLANITO,HOGAR CON VENTA,BRONCE,2025-09-22,...,1,1,reportes_clientes/IndicadoresReport_2025-09-17...,HN2,PR,Preventa Comercial,ZPV,Costas,Tecpan,39
11015239,CENTRO-PACIFICO,CHILAPA-CHILPANCINGO,HV01,HV2,HV0139,1950086116,DULCERIA SOL,ABARROTES Y MISCELANEAS,BRONCE,2025-09-24,...,0,1,reportes_clientes/IndicadoresReport_2025-09-17...,HV2,PR,Preventa Comercial,ZPV,Chilapa-Chilpancingo,Chilpancingo,39


In [8]:
df_mrk_cp['Tiempo de atencion'] = pd.to_timedelta(df_mrk_cp['Tiempo de atencion'], errors='coerce')
df_mrk_cp['Hora de llegada'] = pd.to_datetime(
    df_mrk_cp['Hora de llegada'], format='%H:%M:%S', errors='coerce'
)

# 📊 Crear el pivot con diferentes funciones de agregación
df_pivot_gec = df_mrk_cp.pivot_table(
    index=['Región Comercial_Act 2026','Nombre UO','Jefatura_y','Ruta','Descripción Tipo','Fecha inicio','GEC'],
    aggfunc={
        'Id cliente': 'count',
        'Geoeficiencia': 'sum',
        'Geoefectividad': 'sum',
        'Tiempo de atencion': 'mean',
        'Hora de llegada': 'min',
        'Pedido Omnicanal': 'sum',
    }
)

# 🔹 Convertir timedelta a HH:MM:SS sin días ni decimales
#df_pivot_gec['Tiempo de atencion'] = df_pivot_gec['Tiempo de atencion'].apply(
#    lambda x: str(x).split(' days ')[-1].split('.')[0] if pd.notnull(x) else None
#)

# 🔹 Mostrar solo HH:MM:SS en la hora de llegada
df_pivot_gec['Hora de llegada'] = df_pivot_gec['Hora de llegada'].dt.strftime('%H:%M:%S')

df_pivot_gec


Geoefectividad  \
Región Comercial_Act 2026 Nombre UO Jefatura_y Ruta   Descripción Tipo   Fecha inicio GEC                           
Acapulco                  Cayaco    CJ1        CJ0011 Preventa Comercial 2025-01-02   BRONCE                   15   
                                                                                      CUSTOMIZADO               3   
                                                                                      ORO                      11   
                                                                                      PLATA                    18   
                                                                         2025-01-03   BRONCE                    4   
...                                                                                                           ...   
Puebla Foránea            Tehuacán  TP7        TPM001 Asesor Mayorista   2025-10-25   CUSTOMIZADO              10   
                                                                         2025-10-27   CUSTOMIZADO               9   
                                                                         2025-10-28   CUSTOMIZADO              11   
                                                                         2025-10-29   CUSTOMIZADO               8   
                                                                         2025-10-30   CUSTOMIZADO              11   

                                                                                                   Geoeficiencia  \
Región Comercial_Act 2026 Nombre UO Jefatura_y Ruta   Descripción Tipo   Fecha inicio GEC                          
Acapulco                  Cayaco    CJ1        CJ0011 Preventa Comercial 2025-01-02   BRONCE                  27   
                                                                                      CUSTOMIZADO              4   
                                                                                      ORO                     14   
                                                                                      PLATA                   32   
                                                                         2025-01-03   BRONCE                  30   
...                                                                                                          ...   
Puebla Foránea            Tehuacán  TP7        TPM001 Asesor Mayorista   2025-10-25   CUSTOMIZADO             10   
                                                                         2025-10-27   CUSTOMIZADO              9   
                                                                         2025-10-28   CUSTOMIZADO             11   
                                                                         2025-10-29   CUSTOMIZADO              8   
                                                                         2025-10-30   CUSTOMIZADO             11   

                                                                                                  Hora de llegada  \
Región Comercial_Act 2026 Nombre UO Jefatura_y Ruta   Descripción Tipo   Fecha inicio GEC                           
Acapulco                  Cayaco    CJ1        CJ0011 Preventa Comercial 2025-01-02   BRONCE             10:37:28   
                                                                                      CUSTOMIZADO        10:32:08   
                                                                                      ORO                10:07:36   
                                                                                      PLATA              10:47:29   
                                                                         2025-01-03   BRONCE             08:27:25   
...                                                                                                           ...   
Puebla Foránea            Tehuacán  TP7        TPM001 Asesor Mayorista   2025-10-25   CUSTOMIZADO        08:22:59   
               

In [9]:
df_pivot_gec = df_pivot_gec.reset_index()
df_pivot_gec.info()
df_pivot_gec

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644827 entries, 0 to 644826
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype          
---  ------                     --------------   -----          
 0   Región Comercial_Act 2026  644827 non-null  object         
 1   Nombre UO                  644827 non-null  object         
 2   Jefatura_y                 644827 non-null  object         
 3   Ruta                       644827 non-null  object         
 4   Descripción Tipo           644827 non-null  object         
 5   Fecha inicio               644827 non-null  datetime64[ns] 
 6   GEC                        644827 non-null  object         
 7   Geoefectividad             644827 non-null  int64          
 8   Geoeficiencia              644827 non-null  int64          
 9   Hora de llegada            644827 non-null  object         
 10  Id cliente                 644827 non-null  int64          
 11  Pedido Omnicanal           644827 non-n

,Región Comercial_Act 2026,Nombre UO,Jefatura_y,Ruta,Descripción Tipo,Fecha inicio,GEC,Geoefectividad,Geoeficiencia,Hora de llegada,Id cliente,Pedido Omnicanal,Tiempo de atencion
0,Acapulco,Cayaco,CJ1,CJ0011,Preventa Comercial,2025-01-02,BRONCE,15,27,10:37:28,29,16,0 days 00:00:40.793103448
1,Acapulco,Cayaco,CJ1,CJ0011,Preventa Comercial,2025-01-02,CUSTOMIZADO,3,4,10:32:08,4,3,0 days 00:01:15.750000
2,Acapulco,Cayaco,CJ1,CJ0011,Preventa Comercial,2025-01-02,ORO,11,14,10:07:36,15,12,0 days 00:02:17.533333333
3,Acapulco,Cayaco,CJ1,CJ0011,Preventa Comercial,2025-01-02,PLATA,18,32,10:47:29,32,20,0 days 00:01:15.875000
4,Acapulco,Cayaco,CJ1,CJ0011,Preventa Comercial,2025-01-03,BRONCE,4,30,08:27:25,38,6,0 days 00:00:09.210526315
...,...,...,...,...,...,...,...,...,...,...,...,...,...
644822,Puebla Foránea,Tehuacán,TP7,TPM001,Asesor Mayorista,2025-10-25,CUSTOMIZADO,10,10,08:22:59,11,11,0 days 00:07:11.272727272
644823,Puebla Foránea,Tehuacán,TP7,TPM001,Asesor Mayorista,2025-10-27,CUSTOMIZADO,9,9,07:35:37,9,9,0 days 00:08:30.666666666
644824,Puebla Foránea,Tehuacán,TP7,TPM001,Asesor Mayorista,2025-10-28,CUSTOMIZADO,11,11,07:24:29,11,11,0 days 00:06:35.636363636
644825,Puebla Foránea,Tehuacán,TP7,TPM001,Asesor Mayorista,2025-10-29,CUSTOMIZADO,8,8,06:00:41,9,8,0 days 00:05:06.666666666


In [10]:
df_final_cliente = df_pivot_gec[['Fecha inicio','Región Comercial_Act 2026','Nombre UO','Jefatura_y','Ruta','Descripción Tipo','GEC','Id cliente','Geoeficiencia','Geoefectividad','Hora de llegada','Tiempo de atencion','Pedido Omnicanal']]

df_final_cliente = df_final_cliente.rename(columns={'Jefatura_y': 'Jefatura'})

df_final_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644827 entries, 0 to 644826
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype          
---  ------                     --------------   -----          
 0   Fecha inicio               644827 non-null  datetime64[ns] 
 1   Región Comercial_Act 2026  644827 non-null  object         
 2   Nombre UO                  644827 non-null  object         
 3   Jefatura                   644827 non-null  object         
 4   Ruta                       644827 non-null  object         
 5   Descripción Tipo           644827 non-null  object         
 6   GEC                        644827 non-null  object         
 7   Id cliente                 644827 non-null  int64          
 8   Geoeficiencia              644827 non-null  int64          
 9   Geoefectividad             644827 non-null  int64          
 10  Hora de llegada            644827 non-null  object         
 11  Tiempo de atencion         644827 non-n

In [11]:
df_final_cliente.to_parquet('bdd_mrk_cp_gec.parquet')